### Colorectal Cancer Dataset

In [1]:
# load modules

import sys
sys.path.append(r"C:\Users\35799\Desktop\cookiecutter-analytical-project\biolizard-internship-marios\src")
import pandas as pd
from src import ai_module as ai

In [2]:
# define data folder path and data file names

raw_data_folder = r"C:\Users\35799\Desktop\cookiecutter-analytical-project\biolizard-internship-marios\data\raw"

gene_expression_data = r"Colorectal Cancer Gene Expression Data.csv"
patient_data = r"Colorectal Cancer Patient Data.csv"

In [3]:
# load data

gene_expression_df = ai.data_load(raw_data_folder, gene_expression_data)
patient_df = ai.data_load(raw_data_folder, patient_data)

In [4]:
# combine data into one Pandas DataFrame

gene_expression_df_transposed = gene_expression_df.transpose()
gene_expression_df_transposed.columns = gene_expression_df_transposed.iloc[0]
gene_expression_df_transposed_final = gene_expression_df_transposed.drop("ID_REF", axis=0)

df = pd.merge(patient_df, gene_expression_df_transposed_final, right_index=True, left_on='ID_REF')
df.head()

ID_REF  Age (in years) Dukes Stage  Gender Location  DFS (in months)  \
0  GSM877126            62.0           A    Male     Left            108.0   
1  GSM877127            77.0           B    Male     Left             40.0   
2  GSM877128            66.0           C  Female     Left             49.0   
3  GSM877129            72.0           D  Female     Left             45.0   
4  GSM877130            75.0           C    Male     Left             40.0   

   DFS event  Adj_Radio  Adj_Chem    117_at  1007_s_at   1053_at    121_at  \
0        0.0        1.0       0.0  6.029834   9.468893   7.18709  6.807582   
1        1.0        1.0       0.0  6.344321   9.994204  7.705626   6.89313   
2        0.0        1.0       0.0  5.023127  10.409823  7.934057  7.165602   
3        0.0        1.0       1.0  4.612727  10.596921  7.446374  7.071417   
4        0.0        0.0       1.0  5.865944  10.252547  7.141105  7.103383   

  1255_g_at   1294_at   1316_at   1320_at 1405_i_at   1431_at   1438_at  \
0  3.084591  7.397013  6.525426  3.848253  7.572331    3.7482  6.957488   
1  3.118704  7.163797  6.391195  4.202085  8.295322   3.11434  7.366494   
2  2.984632  7.375224  5.538846  4.074462  7.055579   4.17227  9.190242   
3  2.912851  7.596233  5.989148  3.738205  9.683806  3.625413  7.493741   
4  3.383726  7.278244  5.678236  4.141955  7.360892  3.338311   6.06871   

    1487_at 1494_f_at 1552256_a_at 1552257_a_at 1552258_at 1552261_at  \
0  8.057704   4.31196     7.246198     8.998012    3.91272   4.206709   
1  8.450329  4.257666     8.113613     9.154566   4.223799   3.359484   
2  8.186155  4.390612     7.038909     8.273925   3.988332   3.434758   
3  7.398235  4.308245     7.547753     9.075829    3.88578   4.225294   
4  7.919004  4.188543     7.921648     7.992741   4.960633   3.391169   

  1552263_at 1552264_a_at 1552266_at 1552269_at 1552271_at 1552272_a_at  \
0   6.293766     6.585635   2.837637   3.024119   4.561616     4.371035   
1    7.78168     8.131629   2.408522   2.942956     3.8826     4.264255   
2   5.245886     6.081574   2.800716   2.845071    4.74935     5.269892   
3   5.958728     6.779613   2.659192   3.280929   4.333246     4.509829   
4   6.249177     6.517431   2.556627   2.872373   4.438411     3.997675   

  1552274_at 1552275_s_at 1552276_a_at 1552277_a_at 1552278_a_at 1552279_a_at  \
0   6.120874     4.361976     4.765517     7.101751     3.900317     5.035382   
1   6.112272     4.974505     4.482178      7.39717     4.620965       5.4176   
2   4.948306     4.151721     4.577981     7.800087     4.231365     5.261286   
3   6.325548      4.53578     4.417799     7.025963     4.167651     5.284253   
4   6.070412     5.086089     4.243861     7.723245     4.286986     5.683226   

  1552280_at 1552281_at 1552283_s_at 1552286_at 1552287_s_at 1552288_at  \
0   2.830732   6.470622     3.906776   5.081563     6.837268   3.297914   
1    3.42486     7.5817      4.59077   4.608948     6.190298   3.921998   
2   3.240356   6.405948     5.156768   5.788224     6.788046   3.498495   
3   3.010893   6.372876     6.108916   5.102382      6.38247   3.253702   
4   3.116138   7.471077     3.897945   4.986585     6.793243    3.22251   

  1552289_a_at 1552291_at 1552293_at 1552295_a_at 1552296_at 1552299_at  \
0     4.153212   6.358204   3.360942     5.999121   3.611584   4.041372   
1     5.008631   6.512021   3.815529     5.832778   3.944787   4.022736   
2      3.47574   6.277524   3.473501     6.072805   3.626742   5.108468   
3     4.094617   6.637175   3.831071     6.029997   4.252454   4.464719   
4     3.844001   5.937285   3.699799     6.698944   3.816713   4.554682   

  1552301_a_at 1552302_at 1552303_a_at 1552304_at 1552306_at 1552307_a_at  \
0     3.673284   4.436078     5.600093   3.365764   3.694957     4.147891   
1      3.66859   3.242581     4.561606   3.347428   4.554358     5.184107   
2     3.723708   3.474229     4.794111   3.983271   3.569708     4.485297   
3     3.599279   3.869152     

In [5]:
# gene_id, gene_categorical, gene_continuous = ai.data_info(gene_expression_df, gene_expression_data, 20)
# patient_id, patient_categorical, patient_continuous = ai.data_info(patient_df, patient_data, 20)
df_id, df_cat, df_cont = ai.data_info(df, "Combined Colorectal Cancer Dataset", 20)

DATA FILE:
----------------------------------------------------------------------------------------------------
Combined Colorectal Cancer Dataset
----------------------------------------------------------------------------------------------------


DIMENSIONS:
----------------------------------------------------------------------------------------------------
Entries: 62
Features: 1944
----------------------------------------------------------------------------------------------------


CATEGORICAL FEATURES:
----------------------------------------------------------------------------------------------------
Features     Data Type    Categories & Counts
-----------  -----------  ---------------------------------------------------
Dukes Stage  object       {'A': 16, 'B': 14, 'C': 20, 'D': 12}
Gender       object       {'Female': 14, 'Male': 48}
Location     object       {'Colon': 2, 'Left': 20, 'Rectum': 18, 'Right': 22}
DFS event    float64      {0.0: 25, 1.0: 37}
Adj_Radio    float64 

In [6]:
corr_matrix = ai.correlations(df, type="pearson", printout="matrix")

Correlation Matrix:


,Age (in years),DFS (in months),DFS event,Adj_Radio,Adj_Chem
Age (in years),1.000,-0.118,0.145,0.171,0.056
DFS (in months),-0.118,1.000,-0.456,-0.053,-0.055
DFS event,0.145,-0.456,1.000,-0.177,-0.059
Adj_Radio,0.171,-0.053,-0.177,1.000,-0.192
Adj_Chem,0.056,-0.055,-0.059,-0.192,1.000


In [7]:
corr_matrix = ai.correlations(df, type="pearson", printout="heatmap")

Heatmap:


In [ ]:
# data split

X, y, X_train, y_train, X_test, y_test, X_val, y_val = ai.data_split(df, target="Dukes Stage", random_state=0, method="tt", train_proportion=0.8, test_proportion=0.2, validation_proportion=0.25, stratify="Yes")

In [8]:
# patient_df_treat_na = ai.treat_na(patient_df, patient_id, patient_categorical, patient_continuous, drop_na_rows=False, impute_value=0.5, categorical_imputer="mode", continuous_imputer="median")
df_treat_na = ai.treat_na(df, df_id, df_cat, df_cont, drop_na_rows=False, impute_cutoff=0.5, categorical_imputer="mode", continuous_imputer="median")

In [9]:
df_treat_duplicate = ai.treat_duplicate(df_treat_na, keep_in="first")